In [34]:
import pandas as pd
import numpy as np
from scipy.io import arff
import os
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import scipy
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score, f1_score
import xgboost as xgb
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials
from hyperopt.pyll.base import scope
from sklearn import metrics
from sklearn.model_selection import cross_val_score

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
phone_accel_df = pd.DataFrame()
filenames = os.listdir('wisdm-dataset/arff_files/phone/accel')
filenames.pop(0)
for file in filenames:
    path = "wisdm-dataset/arff_files/phone/accel/" + file
    data = arff.loadarff(path)
    df = pd.DataFrame(data[0])
    print("for file at " + path + "shape is: " + str(df.shape))
    frames = [df, phone_accel_df]
    phone_accel_df = pd.concat(frames, sort = False)

for file at wisdm-dataset/arff_files/phone/accel/data_1600_accel_phone.arffshape is: (321, 93)
for file at wisdm-dataset/arff_files/phone/accel/data_1601_accel_phone.arffshape is: (407, 93)
for file at wisdm-dataset/arff_files/phone/accel/data_1602_accel_phone.arffshape is: (424, 93)
for file at wisdm-dataset/arff_files/phone/accel/data_1603_accel_phone.arffshape is: (409, 93)
for file at wisdm-dataset/arff_files/phone/accel/data_1604_accel_phone.arffshape is: (321, 93)
for file at wisdm-dataset/arff_files/phone/accel/data_1605_accel_phone.arffshape is: (406, 93)
for file at wisdm-dataset/arff_files/phone/accel/data_1606_accel_phone.arffshape is: (321, 93)
for file at wisdm-dataset/arff_files/phone/accel/data_1607_accel_phone.arffshape is: (383, 93)
for file at wisdm-dataset/arff_files/phone/accel/data_1608_accel_phone.arffshape is: (482, 93)
for file at wisdm-dataset/arff_files/phone/accel/data_1609_accel_phone.arffshape is: (306, 93)
for file at wisdm-dataset/arff_files/phone/accel/d

In [4]:
phone_accel_df.shape

(23074, 93)

In [5]:
phone_accel_old_column_names = phone_accel_df.columns

In [6]:
phone_accel_new_column_names = []
for name in phone_accel_old_column_names:
    phone_accel_new_column_names.append(name.replace('"', ''))

In [7]:
replacement_dict = dict(zip(phone_accel_old_column_names, phone_accel_new_column_names))
phone_accel_df = phone_accel_df.rename(columns = replacement_dict)
phone_accel_df.head()

,ACTIVITY,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,Y0,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Z0,Z1,Z2,Z3,Z4,Z5,Z6,Z7,Z8,Z9,XAVG,YAVG,ZAVG,XPEAK,YPEAK,ZPEAK,XABSOLDEV,YABSOLDEV,ZABSOLDEV,XSTANDDEV,YSTANDDEV,ZSTANDDEV,XVAR,YVAR,ZVAR,XMFCC0,XMFCC1,XMFCC2,XMFCC3,XMFCC4,XMFCC5,XMFCC6,XMFCC7,XMFCC8,XMFCC9,XMFCC10,XMFCC11,XMFCC12,YMFCC0,YMFCC1,YMFCC2,YMFCC3,YMFCC4,YMFCC5,YMFCC6,YMFCC7,YMFCC8,YMFCC9,YMFCC10,YMFCC11,YMFCC12,ZMFCC0,ZMFCC1,ZMFCC2,ZMFCC3,ZMFCC4,ZMFCC5,ZMFCC6,ZMFCC7,ZMFCC8,ZMFCC9,ZMFCC10,ZMFCC11,ZMFCC12,XYCOS,XZCOS,YZCOS,XYCOR,XZCOR,YZCOR,RESULTANT,class
0,b'A',0.230,0.235,0.385,0.085,0.055,0.01,0.0,0.0,0.0,0.0,0.170,0.030,0.085,0.360,0.275,0.07,0.01,0.0,0.0,0.0,0.040,0.050,0.035,0.090,0.315,0.205,0.11,0.105,0.03,0.02,-1.29421,2.39763,7.43651,86.3636,89.5238,71.4815,3.79717,4.20119,3.60697,0.466331,0.458116,0.347447,0.682884,0.676843,0.589446,0.465170,0.657497,0.656441,0.654681,0.652221,0.649062,0.645209,0.640664,0.635433,0.629522,0.622937,0.615685,0.607773,0.444171,0.627817,0.626808,0.625128,0.622779,0.619763,0.616083,0.611744,0.606749,0.601105,0.594817,0.587892,0.580338,0.454504,0.642422,0.641390,0.639671,0.637267,0.634181,0.630416,0.625975,0.620864,0.615089,0.608655,0.601569,0.593838,0.395086,-0.200571,0.405282,0.501933,-0.073767,0.223811,11.51220,b'1650'
1,b'A',0.975,0.020,0.005,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.660,0.255,0.075,0.005,0.005,0.00,0.00,0.0,0.0,0.0,0.370,0.280,0.225,0.085,0.040,0.000,0.00,0.000,0.00,0.00,-8.62912,-3.67598,-1.71978,70.0000,65.3333,71.8519,2.97859,2.15444,3.16653,0.255185,0.193401,0.289781,0.505158,0.439773,0.538313,0.437150,0.617892,0.616899,0.615246,0.612934,0.609965,0.606344,0.602073,0.597157,0.591602,0.585414,0.578598,0.571163,0.389856,0.551044,0.550159,0.548684,0.546622,0.543975,0.540745,0.536937,0.532553,0.527599,0.522080,0.516002,0.509371,0.429256,0.606734,0.605760,0.604136,0.601866,0.598951,0.595395,0.591201,0.586374,0.580920,0.574843,0.568151,0.560850,0.794411,0.366282,0.108550,0.235529,0.026102,-0.366805,10.66480,b'1650'
2,b'A',1.000,0.000,0.000,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.555,0.415,0.030,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.440,0.360,0.150,0.045,0.005,0.000,0.00,0.000,0.00,0.00,-8.41993,-3.07394,-2.68363,83.0435,67.2414,79.5455,2.56174,1.71211,2.75633,0.221399,0.152390,0.245360,0.470530,0.390372,0.495338,0.413661,0.584691,0.583752,0.582188,0.580000,0.577191,0.573764,0.569723,0.565071,0.559814,0.553958,0.547509,0.540474,0.331764,0.468933,0.468180,0.466925,0.465171,0.462918,0.460169,0.456928,0.453197,0.448982,0.444285,0.439113,0.433470,0.396997,0.561138,0.560237,0.558735,0.556636,0.553940,0.550651,0.546772,0.542308,0.537263,0.531643,0.525454,0.518702,0.848597,0.559720,0.454855,0.405486,-0.049683,-0.101471,10.11240,b'1650'
3,b'A',0.995,0.005,0.000,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.500,0.400,0.100,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.330,0.350,0.240,0.075,0.005,0.000,0.00,0.000,0.00,0.00,-8.95552,-2.67376,-1.65992,70.3704,76.2500,71.5385,2.65143,1.86928,2.51211,0.230854,0.158261,0.228532,0.480472,0.397820,0.478050,0.424584,0.600131,0.599167,0.597562,0.595316,0.592433,0.588915,0.584767,0.579993,0.574598,0.568587,0.561967,0.554746,0.340917,0.481871,0.481097,0.479807,0.478004,0.475689,0.472865,0.469534,0.465701,0.461369,0.456542,0.451227,0.445429,0.394244,0.557247,0.556352,0.554861,0.552775,0.550098,0.546832,0.542981,0.538547,0.533538,0.527956,0.521810,0.515105,0.752024,0.515116,0.217559,0.143717,0.281892,-0.232531,10.19920,b'1650'
4,b'A',0.990,0.010,0.000,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.380,0.580,0.040,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.485,0.365,0.115,0.025,0.010,0.000,0.00,0.000,0.00,0.00,-8.60716,-2.40619,-2.52186,101.0530,77.3913,105.0000,2.57757,1.22835,2.00075,0.217810,0.113445,0.185843,0.466701,0.336816,0.431095,0.384544,0.543536,0.542662,0.541208,0.539174,0.536563,0.533377,0.529620,0.525296,0.520410,0.514966,0.508971,0.502430,0.303598,0.429122,0.428433,0.427285,0.425679,0.423617,0.421102,0.418136,0.414722,0.410864,0.406567,0.401833,0.396670,0.373903,0.528495,0.527646,0.526232,0.524254,0.521

In [8]:
labelencoder = LabelEncoder()
phone_accel_df['ACTIVITY'] = labelencoder.fit_transform(phone_accel_df['ACTIVITY'])
phone_accel_df.head()

,ACTIVITY,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,Y0,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Z0,Z1,Z2,Z3,Z4,Z5,Z6,Z7,Z8,Z9,XAVG,YAVG,ZAVG,XPEAK,YPEAK,ZPEAK,XABSOLDEV,YABSOLDEV,ZABSOLDEV,XSTANDDEV,YSTANDDEV,ZSTANDDEV,XVAR,YVAR,ZVAR,XMFCC0,XMFCC1,XMFCC2,XMFCC3,XMFCC4,XMFCC5,XMFCC6,XMFCC7,XMFCC8,XMFCC9,XMFCC10,XMFCC11,XMFCC12,YMFCC0,YMFCC1,YMFCC2,YMFCC3,YMFCC4,YMFCC5,YMFCC6,YMFCC7,YMFCC8,YMFCC9,YMFCC10,YMFCC11,YMFCC12,ZMFCC0,ZMFCC1,ZMFCC2,ZMFCC3,ZMFCC4,ZMFCC5,ZMFCC6,ZMFCC7,ZMFCC8,ZMFCC9,ZMFCC10,ZMFCC11,ZMFCC12,XYCOS,XZCOS,YZCOS,XYCOR,XZCOR,YZCOR,RESULTANT,class
0,0,0.230,0.235,0.385,0.085,0.055,0.01,0.0,0.0,0.0,0.0,0.170,0.030,0.085,0.360,0.275,0.07,0.01,0.0,0.0,0.0,0.040,0.050,0.035,0.090,0.315,0.205,0.11,0.105,0.03,0.02,-1.29421,2.39763,7.43651,86.3636,89.5238,71.4815,3.79717,4.20119,3.60697,0.466331,0.458116,0.347447,0.682884,0.676843,0.589446,0.465170,0.657497,0.656441,0.654681,0.652221,0.649062,0.645209,0.640664,0.635433,0.629522,0.622937,0.615685,0.607773,0.444171,0.627817,0.626808,0.625128,0.622779,0.619763,0.616083,0.611744,0.606749,0.601105,0.594817,0.587892,0.580338,0.454504,0.642422,0.641390,0.639671,0.637267,0.634181,0.630416,0.625975,0.620864,0.615089,0.608655,0.601569,0.593838,0.395086,-0.200571,0.405282,0.501933,-0.073767,0.223811,11.51220,b'1650'
1,0,0.975,0.020,0.005,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.660,0.255,0.075,0.005,0.005,0.00,0.00,0.0,0.0,0.0,0.370,0.280,0.225,0.085,0.040,0.000,0.00,0.000,0.00,0.00,-8.62912,-3.67598,-1.71978,70.0000,65.3333,71.8519,2.97859,2.15444,3.16653,0.255185,0.193401,0.289781,0.505158,0.439773,0.538313,0.437150,0.617892,0.616899,0.615246,0.612934,0.609965,0.606344,0.602073,0.597157,0.591602,0.585414,0.578598,0.571163,0.389856,0.551044,0.550159,0.548684,0.546622,0.543975,0.540745,0.536937,0.532553,0.527599,0.522080,0.516002,0.509371,0.429256,0.606734,0.605760,0.604136,0.601866,0.598951,0.595395,0.591201,0.586374,0.580920,0.574843,0.568151,0.560850,0.794411,0.366282,0.108550,0.235529,0.026102,-0.366805,10.66480,b'1650'
2,0,1.000,0.000,0.000,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.555,0.415,0.030,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.440,0.360,0.150,0.045,0.005,0.000,0.00,0.000,0.00,0.00,-8.41993,-3.07394,-2.68363,83.0435,67.2414,79.5455,2.56174,1.71211,2.75633,0.221399,0.152390,0.245360,0.470530,0.390372,0.495338,0.413661,0.584691,0.583752,0.582188,0.580000,0.577191,0.573764,0.569723,0.565071,0.559814,0.553958,0.547509,0.540474,0.331764,0.468933,0.468180,0.466925,0.465171,0.462918,0.460169,0.456928,0.453197,0.448982,0.444285,0.439113,0.433470,0.396997,0.561138,0.560237,0.558735,0.556636,0.553940,0.550651,0.546772,0.542308,0.537263,0.531643,0.525454,0.518702,0.848597,0.559720,0.454855,0.405486,-0.049683,-0.101471,10.11240,b'1650'
3,0,0.995,0.005,0.000,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.500,0.400,0.100,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.330,0.350,0.240,0.075,0.005,0.000,0.00,0.000,0.00,0.00,-8.95552,-2.67376,-1.65992,70.3704,76.2500,71.5385,2.65143,1.86928,2.51211,0.230854,0.158261,0.228532,0.480472,0.397820,0.478050,0.424584,0.600131,0.599167,0.597562,0.595316,0.592433,0.588915,0.584767,0.579993,0.574598,0.568587,0.561967,0.554746,0.340917,0.481871,0.481097,0.479807,0.478004,0.475689,0.472865,0.469534,0.465701,0.461369,0.456542,0.451227,0.445429,0.394244,0.557247,0.556352,0.554861,0.552775,0.550098,0.546832,0.542981,0.538547,0.533538,0.527956,0.521810,0.515105,0.752024,0.515116,0.217559,0.143717,0.281892,-0.232531,10.19920,b'1650'
4,0,0.990,0.010,0.000,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.380,0.580,0.040,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.485,0.365,0.115,0.025,0.010,0.000,0.00,0.000,0.00,0.00,-8.60716,-2.40619,-2.52186,101.0530,77.3913,105.0000,2.57757,1.22835,2.00075,0.217810,0.113445,0.185843,0.466701,0.336816,0.431095,0.384544,0.543536,0.542662,0.541208,0.539174,0.536563,0.533377,0.529620,0.525296,0.520410,0.514966,0.508971,0.502430,0.303598,0.429122,0.428433,0.427285,0.425679,0.423617,0.421102,0.418136,0.414722,0.410864,0.406567,0.401833,0.396670,0.373903,0.528495,0.527646,0.526232,0.524254,0.521715,0.518618,0.

In [9]:
phone_accel_df.pop('class')
phone_accel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23074 entries, 0 to 320
Data columns (total 92 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ACTIVITY   23074 non-null  int32  
 1   X0         23074 non-null  float64
 2   X1         23074 non-null  float64
 3   X2         23074 non-null  float64
 4   X3         23074 non-null  float64
 5   X4         23074 non-null  float64
 6   X5         23074 non-null  float64
 7   X6         23074 non-null  float64
 8   X7         23074 non-null  float64
 9   X8         23074 non-null  float64
 10  X9         23074 non-null  float64
 11  Y0         23074 non-null  float64
 12  Y1         23074 non-null  float64
 13  Y2         23074 non-null  float64
 14  Y3         23074 non-null  float64
 15  Y4         23074 non-null  float64
 16  Y5         23074 non-null  float64
 17  Y6         23074 non-null  float64
 18  Y7         23074 non-null  float64
 19  Y8         23074 non-null  float64
 20  Y9      

In [10]:
phone_accel_df_shuffle = shuffle(phone_accel_df)
phone_accel_df_shuffle.head(10)

,ACTIVITY,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,Y0,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Z0,Z1,Z2,Z3,Z4,Z5,Z6,Z7,Z8,Z9,XAVG,YAVG,ZAVG,XPEAK,YPEAK,ZPEAK,XABSOLDEV,YABSOLDEV,ZABSOLDEV,XSTANDDEV,YSTANDDEV,ZSTANDDEV,XVAR,YVAR,ZVAR,XMFCC0,XMFCC1,XMFCC2,XMFCC3,XMFCC4,XMFCC5,XMFCC6,XMFCC7,XMFCC8,XMFCC9,XMFCC10,XMFCC11,XMFCC12,YMFCC0,YMFCC1,YMFCC2,YMFCC3,YMFCC4,YMFCC5,YMFCC6,YMFCC7,YMFCC8,YMFCC9,YMFCC10,YMFCC11,YMFCC12,ZMFCC0,ZMFCC1,ZMFCC2,ZMFCC3,ZMFCC4,ZMFCC5,ZMFCC6,ZMFCC7,ZMFCC8,ZMFCC9,ZMFCC10,ZMFCC11,ZMFCC12,XYCOS,XZCOS,YZCOS,XYCOR,XZCOR,YZCOR,RESULTANT
87,4,0.000,0.970,0.030,0.000,0.00,0.000,0.00,0.00,0.000,0.000,0.000,0.000,0.0,0.000,0.000,1.000,0.000,0.00,0.00,0.00,0.00,1.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,-0.151996,9.794690,-1.258880,36.4151,32.2951,33.0508,0.062255,0.022872,0.049490,0.005696,0.002086,0.004751,0.075471,0.045672,0.068926,-0.104081,-0.147114,-0.146878,-0.146484,-0.145934,-0.145227,-0.144365,-0.143348,-0.142178,-0.140855,-0.139382,-0.137759,-0.135989,0.343206,0.485107,0.484328,0.483030,0.481215,0.478884,0.476041,0.472688,0.468828,0.464467,0.459609,0.454258,0.448421,0.057287,0.080973,0.080842,0.080626,0.080323,0.079934,0.079459,0.078900,0.078255,0.077527,0.076717,0.075823,0.074849,-0.883652,0.891302,-0.998565,-0.049950,0.359520,0.054988,9.87698
292,16,0.000,0.000,0.000,0.120,0.88,0.000,0.00,0.00,0.000,0.000,0.000,0.000,0.0,1.000,0.000,0.000,0.000,0.00,0.00,0.00,0.00,0.00,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.0,5.179840,3.448410,6.735140,51.3158,52.7778,33.6207,0.127589,0.102453,0.048898,0.011317,0.008817,0.004875,0.106380,0.093901,0.069823,0.255453,0.361072,0.360492,0.359526,0.358175,0.356440,0.354324,0.351828,0.348956,0.345710,0.342093,0.338111,0.333766,0.196979,0.278421,0.277974,0.277229,0.276187,0.274850,0.273218,0.271293,0.269078,0.266575,0.263787,0.260716,0.257365,0.288686,0.408046,0.407390,0.406298,0.404771,0.402811,0.400420,0.397599,0.394353,0.390685,0.386598,0.382097,0.377187,0.998417,0.999308,0.999261,-0.406369,-0.514073,-0.090117,9.17202
560,12,0.030,0.095,0.530,0.335,0.01,0.000,0.00,0.00,0.000,0.000,1.000,0.000,0.0,0.000,0.000,0.000,0.000,0.00,0.00,0.00,0.00,0.03,0.425,0.520,0.015,0.010,0.000,0.000,0.000,0.0,1.696940,-8.969780,2.418180,76.2500,65.5172,60.3226,1.347730,0.723042,1.144910,0.125610,0.086476,0.104627,0.354415,0.294069,0.323461,0.301150,0.425662,0.424978,0.423839,0.422247,0.420202,0.417707,0.414765,0.411378,0.407551,0.403288,0.398593,0.393471,0.372889,0.527063,0.526216,0.524806,0.522833,0.520301,0.517212,0.513569,0.509376,0.504638,0.499359,0.493545,0.487203,0.269238,0.380556,0.379944,0.378926,0.377502,0.375674,0.373444,0.370813,0.367786,0.364364,0.360553,0.356355,0.351776,-0.653931,0.566059,-0.839512,0.312109,-0.061321,0.080221,9.72761
84,3,0.000,0.000,0.000,0.000,0.00,1.000,0.00,0.00,0.000,0.000,0.000,0.000,1.0,0.000,0.000,0.000,0.000,0.00,0.00,0.00,0.00,0.00,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.0,8.506590,1.445730,4.584040,36.7925,30.7937,36.0377,0.074607,0.048297,0.117841,0.006716,0.004447,0.010113,0.081953,0.066689,0.100562,0.323077,0.456656,0.455922,0.454700,0.452992,0.450798,0.448121,0.444965,0.441332,0.437226,0.432653,0.427616,0.422121,0.073560,0.103974,0.103806,0.103528,0.103139,0.102640,0.102030,0.101312,0.100484,0.099550,0.098508,0.097362,0.096111,0.235690,0.333138,0.332603,0.331711,0.330465,0.328864,0.326912,0.324609,0.321959,0.318964,0.315627,0.311953,0.307944,0.998715,0.999143,0.999337,-0.572334,-0.885071,0.566669,9.77228
159,8,1.000,0.000,0.000,0.000,0.00,0.000,0.00,0.00,0.000,0.000,0.000,0.000,0.0,1.000,0.000,0.000,0.000,0.00,0.00,0.00,1.00,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,-6.873570,4.153160,-6.473260,40.8511,38.2000,35.6364,0.028785,0.047126,0.045386,0.002618,0.004013,0.004011,0.051167,0.063352,0.063329,0.292119,0.412897,0.412234,0.411129,0.409584,0.407600,0.405180,0.402326,0.399042,0.395330,0.391194,0.386640,0.381672,0.217395,0.307278,0.306785,0.305962,0.304813,0.303336,0.301535,0.299411,0.296967,0.294204,0.291127,0.287738,0.284040,0.283667,0.400951,0

In [11]:
phone_accel_data = phone_accel_df_shuffle.to_numpy()

In [12]:
X_phone_accel = phone_accel_data[:,1:]
Y_phone_accel = phone_accel_data[:,0]
print(X_phone_accel.shape)
print(Y_phone_accel.shape)

(23074, 91)
(23074,)


In [13]:
x_train, x_test, y_train, y_test = train_test_split(X_phone_accel, Y_phone_accel, test_size = 0.2)

In [14]:
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials
from hyperopt.pyll.base import scope
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier


In [15]:
space = {
    "n_estimators": hp.choice("n_estimators", [100, 150, 200, 250, 300, 350, 400, 450, 500]),
    "max_depth": hp.quniform("max_depth", 1, 15,1),
    "criterion": hp.choice("criterion", ["gini", "entropy"]),
    "min_samples_split": hp.choice("min_samples_split", [2, 3, 4, 5, 6, 7, 8, 9, 10]),
    "max_features": hp.choice("max_features", ["auto","sqrt","log2"]),
}

In [16]:
def hyperparameter_tuning(space):
    clf = RandomForestClassifier(**space,n_jobs=-1)
    acc = cross_val_score(clf, x_train, y_train,scoring="accuracy").mean()
    return {"loss": -acc, "status": STATUS_OK}

In [17]:
trials = Trials()

best = fmin(
    fn=hyperparameter_tuning,
    space = space, 
    algo=tpe.suggest, 
    max_evals=100, 
    trials=trials
)

print("Best: {}".format(best))

100%|██████████████████████████████████████████| 100/100 [2:47:11<00:00, 100.32s/trial, best loss: -0.8554087671308471]
Best: {'criterion': 1, 'max_depth': 15.0, 'max_features': 0, 'min_samples_split': 2, 'n_estimators': 7}


In [18]:
best

{'criterion': 1,
 'max_depth': 15.0,
 'max_features': 0,
 'min_samples_split': 2,
 'n_estimators': 7}

In [19]:
clf = RandomForestClassifier(criterion = 'entropy', max_depth = 15, max_features = 'auto', min_samples_split = 4, n_estimators = 450, n_jobs = -1)

clf.fit(x_train, y_train)

y_train_pred = clf.predict(x_train)
y_test_pred = clf.predict(x_test)

train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print("Train accuracy: ", train_acc)
print("Test accuracy: ", test_acc)

Train accuracy:  0.9989165176878487
Test accuracy:  0.866738894907909


In [20]:
from sklearn.metrics import precision_score

print("average = Nonen: ", precision_score(y_test, y_test_pred, average = None))
print("average = micro: ", precision_score(y_test, y_test_pred, average = 'micro'))
print("average = macro: ", precision_score(y_test, y_test_pred, average = 'macro'))
print("average = weighted: ", precision_score(y_test, y_test_pred, average = 'weighted'))

average = Nonen:  [0.94680851 0.94927536 0.87606838 0.90650407 0.88489209 0.92694064
 0.89922481 0.89539749 0.81531532 0.84140969 0.87401575 0.87815126
 0.75730994 0.81578947 0.82962963 0.92477876 0.89328063 0.73684211]
average = micro:  0.866738894907909
average = macro:  0.8695352163588475
average = weighted:  0.8693043431962947


In [21]:
from sklearn.metrics import recall_score

print("average = Nonen: ", recall_score(y_test, y_test_pred, average = None))
print("average = micro: ", recall_score(y_test, y_test_pred, average = 'micro'))
print("average = macro: ", recall_score(y_test, y_test_pred, average = 'macro'))
print("average = weighted: ", recall_score(y_test, y_test_pred, average = 'weighted'))

average = Nonen:  [0.95017794 0.95970696 0.9030837  0.86100386 0.88489209 0.87124464
 0.89230769 0.84920635 0.78695652 0.80590717 0.79569892 0.86721992
 0.90877193 0.775      0.8358209  0.88185654 0.91497976 0.84677419]
average = micro:  0.866738894907909
average = macro:  0.8661449484819893
average = weighted:  0.866738894907909


### XgBoost

In [25]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [16]:
import xgboost as xgb

In [27]:

space={ 'eta': hp.uniform("eta", 0.001, 0.5),
        'max_depth': hp.quniform("max_depth", 3, 30, 1),
        'gamma': hp.uniform ('gamma', 1,10),
        'reg_alpha' : hp.uniform('reg_alpha', 0,20),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.01,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': hp.quniform('n_estimators', 100, 400, 50),
        'max_delta_step': hp.quniform('max_delta_step', 0, 10, 1),
        'subsample': hp.uniform('subsample', 0.001, 1),
        'colsample_bylevel': hp.uniform('colsample_bylevel', 0.01, 1),
        'scale_pos_weight': hp.uniform('scale_pos_weaight', 0, 1),
    }


In [28]:
def objective(space):
    clf=xgb.XGBClassifier( eta = space['eta'],
                    n_estimators =int(space['n_estimators']), max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = space['reg_alpha'], reg_lambda = space['reg_lambda'],
                    min_child_weight=int(space['min_child_weight']), colsample_bytree=int(space['colsample_bytree']), 
                    max_delta_step = space['max_delta_step'], subsample = space['subsample'],
                    colsample_bylevel = space['colsample_bylevel'], scale_pos_weight = space['scale_pos_weight'],
                    objective = 'multi:softmax')
    
    evaluation = [( x_train, y_train), ( x_test, y_test)]
    
    clf.fit(x_train, y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

    pred = clf.predict(x_test)
    accuracy = accuracy_score(y_test, pred)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

In [29]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

[03:21:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576:                   
Parameters: { "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


SCORE:                                                                                                                 
0.6266522210184182                                                                                                     
[03:23:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576:                   
Parameters: { "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core,

SCORE:                                                                                                                 
0.6680390032502709                                                                                                     
[03:45:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576:                   
Parameters: { "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


SCORE:                                                                                                                 
0.6496208017334778                                                                                                     
[03:47:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.

SCORE:                                                                                                                 
0.5696641386782232                                                                                                     
[04:07:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576:                   
Parameters: { "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


SCORE:                                                                                                                 
0.618851570964247                                                                                                      
[04:09:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.

SCORE:                                                                                                                 
0.5453954496208018                                                                                                     
[04:25:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576:                   
Parameters: { "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


SCORE:                                                                                                                 
0.5683640303358614                                                                                                     
[04:27:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.

SCORE:                                                                                                                 
0.21083423618634886                                                                                                    
[04:35:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576:                   
Parameters: { "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


SCORE:                                                                                                                 
0.6593716143011917                                                                                                     
[04:36:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.

SCORE:                                                                                                                 
0.657854821235103                                                                                                      
[04:51:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576:                   
Parameters: { "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


SCORE:                                                                                                                 
0.6602383531960997                                                                                                     
[04:52:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.

SCORE:                                                                                                                 
0.5889490790899241                                                                                                     
[05:05:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576:                   
Parameters: { "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


SCORE:                                                                                                                 
0.5958829902491874                                                                                                     
[05:06:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.

SCORE:                                                                                                                 
0.618851570964247                                                                                                      
[05:14:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576:                   
Parameters: { "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


SCORE:                                                                                                                 
0.5778981581798484                                                                                                     
[05:15:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.

In [30]:
best_hyperparams

{'colsample_bylevel': 0.4826970701942182,
 'colsample_bytree': 0.4089965172166954,
 'eta': 0.4084770396313456,
 'gamma': 1.426537452250675,
 'max_delta_step': 1.0,
 'max_depth': 18.0,
 'min_child_weight': 4.0,
 'n_estimators': 400.0,
 'reg_alpha': 3.0799688923230546,
 'reg_lambda': 0.3680269397122049,
 'scale_pos_weight': 0.7013424352411445,
 'subsample': 0.8955483204360999}

In [31]:
clf = xgb.XGBClassifier(eta = 0.4, colsample_bytree = 0.409, gamma = 1.43, max_depth = 18, min_child_weight = 4, 
                        n_estimators = 400, reg_alpha = 3.08, reg_lambda = 0.368, 
                        colsample_bylevel = 0.483, max_delta_step = 1, scale_pos_weight = 0.7, subsample = 0.89, 
                        objective = 'multi:softmax')

clf.fit(x_train, y_train)

y_train_pred = clf.predict(x_train)
y_test_pred = clf.predict(x_test)

train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print("Train accuracy: ", train_acc)
print("Test accuracy: ", test_acc)

[11:04:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[11:04:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Train accuracy:  0.9903570074218538
Test accuracy:  0.8680390032502708


In [32]:
print("average = Nonen: ", precision_score(y_test, y_test_pred, average = None))
print("average = micro: ", precision_score(y_test, y_test_pred, average = 'micro'))
print("average = macro: ", precision_score(y_test, y_test_pred, average = 'macro'))
print("average = weighted: ", precision_score(y_test, y_test_pred, average = 'weighted'))

average = Nonen:  [0.93165468 0.92951542 0.89316239 0.88492063 0.875      0.94372294
 0.91111111 0.88446215 0.82677165 0.77727273 0.83137255 0.82644628
 0.84638554 0.80952381 0.85454545 0.9137931  0.91532258 0.78947368]
average = micro:  0.8680390032502708
average = macro:  0.8691364841356655
average = weighted:  0.8693429462206331


In [33]:
print("average = Nonen: ", recall_score(y_test, y_test_pred, average = None))
print("average = micro: ", recall_score(y_test, y_test_pred, average = 'micro'))
print("average = macro: ", recall_score(y_test, y_test_pred, average = 'macro'))
print("average = weighted: ", recall_score(y_test, y_test_pred, average = 'weighted'))

average = Nonen:  [0.96282528 0.95475113 0.9047619  0.81985294 0.88537549 0.88259109
 0.9010989  0.81918819 0.84677419 0.8028169  0.828125   0.85106383
 0.91830065 0.80952381 0.83038869 0.86530612 0.86641221 0.87548638]
average = micro:  0.8680390032502708
average = macro:  0.8680357074496906
average = weighted:  0.8680390032502708


In [35]:
print("average = Nonen: ", f1_score(y_test, y_test_pred, average = None))
print("average = micro: ", f1_score(y_test, y_test_pred, average = 'micro'))
print("average = macro: ", f1_score(y_test, y_test_pred, average = 'macro'))
print("average = weighted: ", f1_score(y_test, y_test_pred, average = 'weighted'))

average = Nonen:  [0.94698355 0.94196429 0.89892473 0.85114504 0.88015717 0.91213389
 0.90607735 0.85057471 0.83665339 0.78983834 0.8297456  0.83857442
 0.88087774 0.80952381 0.84229391 0.88888889 0.89019608 0.8302583 ]
average = micro:  0.8680390032502708
average = macro:  0.8680450665388402
average = weighted:  0.8681313564520392
